In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from dataclasses import dataclass
from typing import Tuple
import random
import artifact
import sampler
import char

from IPython import display

In [2]:
a1 = artifact.make_arti()
a2 = artifact.make_arti()
a1, a2

(ATK% Sands@(HP%:58>198, DEF:19>19, CD:0>156, EM:23>23),
 CR Hat@(HP:239>508, DEF%:66>212, ATK:19>37, CD:70>124))

In [3]:
s = char.Stats()

In [5]:
a1 + a2

[198 508 212  19 466  37 311 280   0  23   0   0   0   0   0   0   0   0
   0]

In [47]:
%timeit artifact.make_arti()

137 µs ± 13.3 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [1099]:
slotnames = ['Flower', 'Feath', 'Sands', 'Cup', 'Circ']
# statnames = ['CR', 'CD', 'ER', 'EM', 'ATK%', 'ATK', 'HP%', 'HP', 'DEF%', 'DEF', 'Heal',
statnames = ['HP%', 'HP', 'DEF%', 'DEF', 'ATK%', 'ATK', 'CR', 'CD', 'ER', 'EM', 'Heal',
          'Phys', 'Hydro', 'Pyro', 'Cryo', 'Electro', 'Anemo', 'Geo', 'Dendro']
statmap = {n: i for i, n in enumerate(statnames)}